# Compiling the post mass transfer systems that contain a Be star

 Be+sdOB binaries are particularly valuable for studying mass transfer, as they represent clear examples of past binary interaction

In [65]:
import numpy as np
import pandas as pd
import re
import subprocess


## Papers of interest :

- **Lechien + 2025** compute the mass transfer efficiency "Binary stars take what they get: Evidence for Efficient Mass Transfer from Stripped Stars with Rapidly Rotating Companion" [2025arXiv250514780L](https://ui.adsabs.harvard.edu/abs/2025arXiv250514780L/abstract)



In [25]:
### READING the compiled table from Lechien et al. 2025

# Read the LaTeX file
with open("../data/Lechien25_Table1.tex", "r") as f:
    lines = f.readlines()

# Find the start and end of the tabular environment
table_lines = []
in_table = False
for line in lines:
    if r"\begin{tabular}" in line:
        in_table = True
        continue
    if r"\end{tabular}" in line:
        break
    if in_table:
        table_lines.append(line.strip())

# Filter out lines we don't need
table_rows = [line for line in table_lines if not line.startswith("\\") and line]

# Now split and clean each row
cleaned_rows = []
for row in table_rows:
    row = row.split("&")
    row = [re.sub(r"\\\w+", "", col).strip() for col in row]  # remove LaTeX commands like \cite
    row = [re.sub(r"\$|\\", "", col) for col in row]          # remove $ signs and backslashes
    row = [col.replace("pm", "±").replace("{", "").replace("}", "") for col in row]
    cleaned_rows.append(row)

# Create a DataFrame (skip header rows if needed)
header = [
    "HD", "Name", "M_Be", "M_sdOB", "Period", "Reference", "Tier",
    "beta_min", "beta_range"
]

# Trim extra rows that aren't data
data_rows = [r for r in cleaned_rows if len(r) == len(header)]

Lechien25 = pd.DataFrame(data_rows, columns=header)

# Remove the first 2 rows (which arent data)
Lechien25 = Lechien25.iloc[2:].reset_index(drop=True)


### Put the periods in the right format

In [54]:
##############################
# Periods
Lechien25[['Period_val',  'Period_err']] = Lechien25['Period'].str.split(expand=True) 
Period_val = Lechien25['Period'].str.split(expand=True)  # Extracting Period values and errors
# Period_values = [Period_val[1][:], Period_val[0][:], Period_val[1][:]] # Create a lis to be added to the DataFrame
Period_values = [Lechien25['Period_err'], Lechien25['Period_val'], Lechien25['Period_err']] # Create a lis to be added to the DataFrame
# print(Period_values)

### Put the masses in the right format

In [79]:
##############################
# Masses
def parse_mass_entry(entry):
    entry = str(entry).strip()
    
    if '--' in entry:
        # Range case: "10--13"
        try:
            low, high = map(float, entry.split('--'))
            median = (low + high) / 2
            err = (high - low) / 2
            return [err, median, err, '']
        except ValueError:
            return None
    elif ' ' in entry:
        # Symmetric error case: "9.6 0.3"
        try:
            value, err = map(float, entry.split())
            return [err, value, err, '']
        except ValueError:
            return None
    else:
        # Single number, no error
        try:
            value = float(entry)
            return [None, value, None,'']
        except ValueError:
            return None

Lechien25['M_Be_parsed'] = Lechien25['M_Be'].apply(parse_mass_entry)
Lechien25['M_sdOB_parsed'] = Lechien25['M_sdOB'].apply(parse_mass_entry)

# Show the result
display(Lechien25)

# display(Lechien25['M_Be', 'M_sdOB'].)

,HD,Name,M_Be,M_sdOB,Period,Reference,Tier,beta_min,beta_range,Period_val,Period_err,M_Be_parsed,M_sdOB_parsed
0,58978,FY CMa,10--13,1.1--1.5,37.253 0.007,peters_detection_2008,-,0.57,0.81 - 1.00,37.253,0.007,"[1.5, 11.5, 1.5, ]","[0.19999999999999996, 1.3, 0.19999999999999996, ]"
1,200120,59 Cyg,6.3--9.4,0.62--0.91,28.1871 0.0011,peters_far-ultraviolet_2013,-,0.50,0.75 - 1.00,28.1871,0.0011,"[1.5500000000000003, 7.85, 1.5500000000000003, ]","[0.14500000000000002, 0.765, 0.145000000000000..."
2,10516,Per,9.6 0.3,1.2 0.2,126.6982 0.0035,mourard_spectral_2015,**,0.36,0.56 - 1.00,126.6982,0.0035,"[0.3, 9.6, 0.3, ]","[0.2, 1.2, 0.2, ]"
3,55606,-,6.0--6.6,0.83--0.90,93.76 0.02,chojnowski_remarkable_2018,-,0.06,0.23 - 0.83,93.76,0.02,"[0.2999999999999998, 6.3, 0.2999999999999998, ]","[0.03500000000000003, 0.865, 0.035000000000000..."
4,109387,Dra,3.65 0.48,0.426 0.043,61.5496 0.0058,klement_dynamical_2022,*,0.05,0.25 - 0.83,61.5496,0.0058,"[0.48, 3.65, 0.48, ]","[0.043, 0.426, 0.043, ]"
5,113120,LS Mus,10.1 2.2,1.43 0.31,181.54 0.11,wang_orbital_2023,-,0.27,0.46 - 1.00,181.54,0.11,"[2.2, 10.1, 2.2, ]","[0.31, 1.43, 0.31, ]"
6,137387,Aps,11.8 1.0,1.60 0.14,192.1 0.1,wang_orbital_2023,-,0.41,0.62 - 1.00,192.1,0.1,"[1.0, 11.8, 1.0, ]","[0.14, 1.6, 0.14, ]"
7,152478,V846 Ara,6.5 1.3,0.53 0.11,236.50 0.18,wang_orbital_2023,-,0.64,0.94 - 1.00,236.50,0.18,"[1.3, 6.5, 1.3, ]","[0.11, 0.53, 0.11, ]"
8,157042,Ara,10.5 2.9,1.06 0.29,176.17 0.04,wang_orbital_2023,-,0.64,0.88 - 1.00,176.17,0.04,"[2.9, 10.5, 2.9, ]","[0.29, 1.06, 0.29, ]"
9,41335,HR 2142,17.6 5.7,1.03 0.22,80.8733 0.0044,klement_chara_2024,*,(1.00),(1.00 - 1.00),80.8733,0.0044,"[5.7, 17.6, 5.7, ]","[0.22, 1.03, 0.22, ]"


### Get RA and DEC for all these systems

In [ ]:
# Get RA and DEC using the Get_Coords function
display(Lechien25['HD'].values)

targets = ['HD58978','HD200120','HD10516','HD55606',
'HD109387','HD113120','HD137387','HD152478','HD157042',
'HD41335','HD161306','HD183537','HD191610','HD194335','HD200310','HD167128']

# Run the script and capture output
result = subprocess.run(["python3", "Get_Coords_From_SIMBAD.py"] + targets, capture_output=True, text=True)

# Get the printed output as a string
RA_DEC = result.stdout



array(['58978', '200120', '10516', '55606', '109387', '113120', '137387',
       '152478', '157042', '41335', '161306', '183537', '191610',
       '194335', '200310', '167128'], dtype=object)

H


In [76]:
# Print the output
print(RA_DEC)


HD__58978 = {
    "System Name": 'HD  58978',
    "RA":  [0.00000785, 111.747845, 0.00000785],
    "Dec": [0.00001247, -23.086025, 0.00001247],
}

*_F01_CYG = {
    "System Name": '* f01 Cyg',
    "RA":  [0.00011299, 314.956476, 0.00011299],
    "Dec": [0.00008039, 47.520934, 0.00008039],
}

*_PHI_PER = {
    "System Name": '* phi Per',
    "RA":  [0.00006187, 25.915158, 0.00006187],
    "Dec": [0.00003664, 50.688731, 0.00003664],
}

HD__55606 = {
    "System Name": 'HD  55606',
    "RA":  [0.00000581, 108.392085, 0.00000581],
    "Dec": [0.00000536, -2.077528, 0.00000536],
}

*_KAP_DRA = {
    "System Name": '* kap Dra',
    "RA":  [0.00018685, 188.370597, 0.00018685],
    "Dec": [0.00008244, 69.788239, 0.00008244],
}

HD_113120 = {
    "System Name": 'HD 113120',
    "RA":  [0.00005351, 195.772233, 0.00005351],
    "Dec": [0.00001806, -71.475734, 0.00001806],
}

*_KAP01_APS = {
    "System Name": '* kap01 Aps',
    "RA":  [0.00004868, 232.878426, 0.00004868],
    "Dec": [0.00001608, 

- **Wang + 2021** "The Detection and Characterization of Be+sdO Binaries from HST/STIS FUV Spectroscopy" [2021AJ....161..248W](https://ui.adsabs.harvard.edu/abs/2021AJ....161..248W/abstract)

- **Klement + 2024** The CHARA Array Interferometric Program on the Multiplicity of Classical Be Stars: New Detections and Orbits of Stripped Subdwarf Companions  [2024ApJ...962...70K](https://ui.adsabs.harvard.edu/abs/2024ApJ...962...70K/abstract)

- **Klement + 2025**  VLTI/GRAVITY enables the determination of the first dynamical masses of a classical Be + stripped and bloated pre-subdwarf binary [2025A&A...694A.208K](https://ui.adsabs.harvard.edu/abs/2025A%26A...694A.208K/abstract)



In [ ]:
# Define columns, with key quantities stored as lists: [err-, value, err+]
columns = [
    "System Name", "RA", "Dec", "Period", "Eccentricity",
    "M1", "M2", "q", "Mass Function",
    "Type1", "Type2", "Detection Method", "Reference", "Notes"
]

# Initialize empty DataFrame
observations_df = pd.DataFrame(columns=columns)

# Define helper function using [err-, value, err+] triplets
def add_observation(df, system_name,
                    ra, dec, period, ecc,
                    m1, m1_min_bool, m2, m1_min_bool, q, mass_func,
                    type1, type2, method, reference, notes=""):
    # use np.inf if it is a lower limit (0 if it is an upper limit)
    new_row = {
        "System Name": system_name,
        "RA": ra,                           # [err-, value, err+]
        "Dec": dec,                         # [err-, value, err+]

        "Period": period,                   # day [err-, value, err+]
        "Eccentricity": ecc,                # [err-, value, err+]
        "M1": m1,                           # Accretor star [err-, value, err+, lower/upper limit? ]
        "M2": m2,                           # Donor (post MT 1) [err-, value, err+, lower/upper limit? ]
        "Mass Function": mass_func,         # [err-, value, err+]
        "q": q,                            # M2/M1 = donor/accretor [err-, value, err+ ]

        "Type1": type1,                     # ["MS", "WD", "NS", "BH" "RG", "O", "B" ]
        "Type2": type2,                     # ["MS", "WD", "NS", "BH" "RG", "O", "B" ]
        "Detection Method": method,         # list of strings ["Xray", "RV"= Radial velocity, "EB"=Eclipsing binary, "AB" = Astrometric binary, "Other"]  
        "Reference": reference,             # ADS Bibcode    
        "Notes": notes
    }
    return pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)






In [ ]:
# Example: add an entry
observations_df = add_observation(
    observations_df,
    "Example system",
    [0.005, 150.025, 0.005],        # RA
    [0.004, -45.123, 0.004],        # Dec

    [0.2, 12.5, 0.3, ''],               # Period in days
    [0.02, 0.30, 0.03, ''],             # Eccentricity
    [0.5, 5.0, 0.5],           # M1 upper limit (i.e upper limit of 5 +- 0.5 Msun)
    True
    [0.2, 2.5, 0.3, ''],                # M2 in Msun
    [0.03, 0.25, 0.04, ''],             # q (mass ratio)
    [0.01, 1e-3, 0.02, ''],             # Mass Function
    
    "MS",                            # Type1
    "WD",                            # Type2
    ["RV", "EB"],                    # Detection Method
    "2025arXiv250514780L",           # Reference (use Ads Bibcode)
    "This is an example system ",    # Notes
)